# Checking the quality

In this notebook, we will check if there are wrong formatting or any issue in the outputs

## Libraries

In [9]:
import pandas as pd
import re 
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

## Global variables 

In [10]:
ROOT = Path("../../data")
subset_name = f"subset_{0}"
annotator = "mistral"
model_name = "gpt-4o-mini"
CHECKPOINT_parquet = ROOT / "cleaned_annotation" / f"{subset_name}_{model_name}_cleaned_{annotator}.parquet"

In [11]:
df = pd.read_parquet(CHECKPOINT_parquet)
# Copy the original dataframe file as a backup
backup_file = CHECKPOINT_parquet.with_name(CHECKPOINT_parquet.stem + "_backup" + CHECKPOINT_parquet.suffix)
if not backup_file.exists():
    df.to_parquet(backup_file)
    print(f"Backup created: {backup_file}")
df = df.dropna()
df

Backup created: ../../data/cleaned_annotation/subset_0_gpt-4o-mini_cleaned_mistral_backup.parquet


,msg_id,user,content,topic,deleted,banned,hour,explication,ton,intention,...,labels,toxicite_score,categorie_list,categorie_justification,labels_list,labels_justification,note,note_justification,certitude,annotation
0,anon_msg_9198a73c84ed,anon_user_079568ca56,osef le drapeau fr en allemagne a besoin de so...,anon_topic_de7347d6,1,1,5,Le message indique que le drapeau français en ...,"Tons identifiés : Apathique (80%), Critique (6...",1. Exprimer une opinion (degré de certitude : ...,...,Labels : S0/H0/V0/R0/A2/I2 \nJustification : ...,Note : 2/10 \nJustification : Expression d'ap...,[Évocation ambiguë d’un propos toxique],Le message critique une situation sans être cl...,"[S0, H0, V0, R0, A2, I2]",Le message critique le manque de soutien sans ...,2/10,Expression d'apathie et critique légère sans a...,élevée,Plutôt oui
1,anon_msg_0ed0dcd30a95,anon_user_e8a049aa62,Ça rage sur les chad novax au sang 10/10,anon_topic_e697e741,1,0,10,Le message évoque une frustration ou une colèr...,"Tons identifiés : Indigné (80%), Moqueur (70%)...","Intentions identifiées : Critiquer (80%), Se m...",...,Labels : S0/H2/V0/R0/A2/I2 \nJustification : ...,Note : 4/10 \nJustification : Moquerie appuyé...,"[Évocation d’un propos toxique avec adhésion, ...","Le message se moque des ""chad novax"" en exprim...","[S0, H2, V0, R0, A2, I2]","Le message se moque d'un groupe spécifique, av...",4/10,Moquerie appuyée et condescendance envers un g...,élevée,Plutôt oui
2,anon_msg_b1641d66622a,anon_user_4fe7b2f2fe,"Pas moi, je ne connais pas de mégane",anon_topic_a64f2ab6,1,0,5,Le message exprime que l'auteur ne connaît pas...,"Tons identifiés : Neutre (certitude élevée), D...","Intentions identifiées : Informer, Exprimer un...",...,Labels : S0/H0/V0/R0/A0/I0 \nJustification : ...,"Note : 0/10 \nJustification : Message neutre,...",[],"Le message est neutre et factuel, sans indicat...","[S0, H0, V0, R0, A0, I0]","Le message est factuel et neutre, sans contenu...",0/10,"Message neutre, factuel et sans connotation ho...",élevée,"Oui, tout à fait"
3,anon_msg_8c19931d9a0b,anon_user_ab4654f2c9,l'été arrive et l'odeur des pieds puants des k...,anon_topic_e6ef1ef7,1,0,5,Le message évoque l'arrivée de l'été et décrit...,"Tons identifiés : Ironique (80%), Humoristique...",Les intentions identifiées sont : Exprimer une...,...,Labels : S0/H2/V0/R0/A2/I2 \nJustification : ...,Note : 4/10 \nJustification : Moquerie et mép...,"[Humour à double lecture, Évocation ambiguë d’...",Le message utilise l'humour pour critiquer un ...,"[S0, H2, V0, R0, A2, I2]","Pas d'allusion sexuelle, mais moquerie stéréot...",4/10,Moquerie et mépris stéréotypés avec intention ...,moyenne,Plutôt oui
4,anon_msg_80440252f1ca,anon_user_a034768ac3,Si tu es épanoui et que tu vis bien c'est très...,anon_topic_d956e559,1,1,5,Le message indique que si quelqu'un se sent ép...,Tons identifiés : Bienveillant (certitude élev...,"Intentions identifiées : Valoriser, Informer, ...",...,Labels : S0/H0/V0/R0/A0/I0 \nJustification : ...,Note : 0/10 \nJustification : Message bienvei...,[],Le message est clairement bienveillant et valo...,"[S0, H0, V0, R0, A0, I0]","Message bienveillant et factuel, sans allusion...",0/10,"Message bienveillant et positif, valorisant le...",élevée,"Oui, tout à fait"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21399,anon_msg_f0994549b621,anon_user_d1536ba4ce,El famoso ovni aliens avec un parachute,anon_topic_8d425fc1,1,1,3,- **Sujet du message** : Mention d'un objet vo...,"Tons perçus : Neutre, Humoristique\n\nJustific...",1. **Informer** (certitude : élevée) \n Jus...,...,Labels : S0/H0/V0/R3/A0/I1 \nJustification : ...,Note : 1/10 \nJustification : Le message est ...,[Humour à double lecture],Le message utilise une combinaison absurde pou...,"[S0, H0, V0, R3, A0, I1]",Le message est humoristique avec une référence...,1/10,"Le message est humoristique, neutre, sans cont...",élevée,Plutôt oui
21400,anon_msg_4c3a0fb9028d,anon_user_d0fa84cce4,"""Jud

## Utility functions

In [12]:
def extract_annotation(text):
    # if it contains "plutôt oui", "plutôt non", "oui, tout à fait", "non, pas du tout"
    if "plutôt oui" in text.lower():
        return "plutôt oui"
    elif "plutôt non" in text.lower():
        return "plutôt non"
    elif "oui, tout à fait" in text.lower() or "oui tout à fait" in text.lower():
        return "oui, tout à fait"
    elif "non, pas du tout" in text.lower() or "non pas du tout" in text.lower():
        return "non, pas du tout"
    else:
        return "other"

## Quality check

In [13]:
df['annotation'] = df['annotation'].apply(lambda x: extract_annotation(x))
df = df[df['annotation'] != "other"]

In [14]:
df.to_parquet(CHECKPOINT_parquet, index=False)

In [15]:
df

,msg_id,user,content,topic,deleted,banned,hour,explication,ton,intention,...,labels,toxicite_score,categorie_list,categorie_justification,labels_list,labels_justification,note,note_justification,certitude,annotation
0,anon_msg_9198a73c84ed,anon_user_079568ca56,osef le drapeau fr en allemagne a besoin de so...,anon_topic_de7347d6,1,1,5,Le message indique que le drapeau français en ...,"Tons identifiés : Apathique (80%), Critique (6...",1. Exprimer une opinion (degré de certitude : ...,...,Labels : S0/H0/V0/R0/A2/I2 \nJustification : ...,Note : 2/10 \nJustification : Expression d'ap...,[Évocation ambiguë d’un propos toxique],Le message critique une situation sans être cl...,"[S0, H0, V0, R0, A2, I2]",Le message critique le manque de soutien sans ...,2/10,Expression d'apathie et critique légère sans a...,élevée,plutôt oui
1,anon_msg_0ed0dcd30a95,anon_user_e8a049aa62,Ça rage sur les chad novax au sang 10/10,anon_topic_e697e741,1,0,10,Le message évoque une frustration ou une colèr...,"Tons identifiés : Indigné (80%), Moqueur (70%)...","Intentions identifiées : Critiquer (80%), Se m...",...,Labels : S0/H2/V0/R0/A2/I2 \nJustification : ...,Note : 4/10 \nJustification : Moquerie appuyé...,"[Évocation d’un propos toxique avec adhésion, ...","Le message se moque des ""chad novax"" en exprim...","[S0, H2, V0, R0, A2, I2]","Le message se moque d'un groupe spécifique, av...",4/10,Moquerie appuyée et condescendance envers un g...,élevée,plutôt oui
2,anon_msg_b1641d66622a,anon_user_4fe7b2f2fe,"Pas moi, je ne connais pas de mégane",anon_topic_a64f2ab6,1,0,5,Le message exprime que l'auteur ne connaît pas...,"Tons identifiés : Neutre (certitude élevée), D...","Intentions identifiées : Informer, Exprimer un...",...,Labels : S0/H0/V0/R0/A0/I0 \nJustification : ...,"Note : 0/10 \nJustification : Message neutre,...",[],"Le message est neutre et factuel, sans indicat...","[S0, H0, V0, R0, A0, I0]","Le message est factuel et neutre, sans contenu...",0/10,"Message neutre, factuel et sans connotation ho...",élevée,"oui, tout à fait"
3,anon_msg_8c19931d9a0b,anon_user_ab4654f2c9,l'été arrive et l'odeur des pieds puants des k...,anon_topic_e6ef1ef7,1,0,5,Le message évoque l'arrivée de l'été et décrit...,"Tons identifiés : Ironique (80%), Humoristique...",Les intentions identifiées sont : Exprimer une...,...,Labels : S0/H2/V0/R0/A2/I2 \nJustification : ...,Note : 4/10 \nJustification : Moquerie et mép...,"[Humour à double lecture, Évocation ambiguë d’...",Le message utilise l'humour pour critiquer un ...,"[S0, H2, V0, R0, A2, I2]","Pas d'allusion sexuelle, mais moquerie stéréot...",4/10,Moquerie et mépris stéréotypés avec intention ...,moyenne,plutôt oui
4,anon_msg_80440252f1ca,anon_user_a034768ac3,Si tu es épanoui et que tu vis bien c'est très...,anon_topic_d956e559,1,1,5,Le message indique que si quelqu'un se sent ép...,Tons identifiés : Bienveillant (certitude élev...,"Intentions identifiées : Valoriser, Informer, ...",...,Labels : S0/H0/V0/R0/A0/I0 \nJustification : ...,Note : 0/10 \nJustification : Message bienvei...,[],Le message est clairement bienveillant et valo...,"[S0, H0, V0, R0, A0, I0]","Message bienveillant et factuel, sans allusion...",0/10,"Message bienveillant et positif, valorisant le...",élevée,"oui, tout à fait"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21399,anon_msg_f0994549b621,anon_user_d1536ba4ce,El famoso ovni aliens avec un parachute,anon_topic_8d425fc1,1,1,3,- **Sujet du message** : Mention d'un objet vo...,"Tons perçus : Neutre, Humoristique\n\nJustific...",1. **Informer** (certitude : élevée) \n Jus...,...,Labels : S0/H0/V0/R3/A0/I1 \nJustification : ...,Note : 1/10 \nJustification : Le message est ...,[Humour à double lecture],Le message utilise une combinaison absurde pou...,"[S0, H0, V0, R3, A0, I1]",Le message est humoristique avec une référence...,1/10,"Le message est humoristique, neutre, sans cont...",élevée,plutôt oui
21400,anon_msg_4c3a0fb9028d,anon_user_d0fa84cce4,"""Jud